# Set up

In [3]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Chi Square test

In [7]:
Labels_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

def get_data_chi2test(diseases, types):
    if (types == 'race'):
        df = [[0, 0], [0, 0], [0, 0]]
    elif (types == 'gender'):
        df = [[0, 0], [0, 0]] 
    elif (types == 'age'):
        df = [[0, 0], [0, 0], [0, 0], [0, 0]]
    else:
        print('No match category')
        return

    filename = ['data/mimic_train.tfrecords', 'data/mimic_test.tfrecords', 'data/mimic_val.tfrecords']    
        
    raw_dataset = tf.data.TFRecordDataset(filename)
    for raw_record in raw_dataset:

        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        
        if (example.features.feature[diseases].float_list.value[0] == 1):
            label = 1
        else:
            label = 0
            
        if (types == 'race'):
            race = example.features.feature['race'].int64_list.value[0]
            if (race == 4):
                race = 2
            df[race][label] += 1  
        elif (types == 'gender'):
            gender = example.features.feature['gender'].int64_list.value[0]
            df[gender][label] += 1
        else:
            age = example.features.feature['age'].int64_list.value[0]
            if (age > 1):
                age -= 1
            df[age][label] += 1
             
    return np.array(df)

In [ ]:
from scipy.stats import chi2_contingency, chisquare

for i in Labels_diseases:
    obs = get_data_chi2test(i, 'age')
    
    chi2, p, dof, ex = chi2_contingency(obs)

    (chi2, p) == chisquare(obs.ravel(), f_exp=ex.ravel(),
                                 ddof=obs.size - 1 - dof)
    print(i, p)

In [ ]:
from scipy.stats import chi2_contingency, chisquare

for i in Labels_diseases:
    obs = get_data_chi2test(i, 'gender')

    chi2, p, dof, ex = chi2_contingency(obs)

    (chi2, p) == chisquare(obs.ravel(), f_exp=ex.ravel(),
                                 ddof=obs.size - 1 - dof)
    print(i, p)

In [ ]:
from scipy.stats import chi2_contingency, chisquare

for i in Labels_diseases:
    obs = get_data_chi2test(i, 'race')

    chi2, p, dof, ex = chi2_contingency(obs)

    (chi2, p) == chisquare(obs.ravel(), f_exp=ex.ravel(),
                                 ddof=obs.size - 1 - dof)
    print(i, p)

# Permutation test

In [25]:
def get_data_permutaitontest(data='test', types='race'):
    x = []
    y = []

    if (data=='train'):
        filename = ['data/mimic_train.tfrecords']
    else:
        filename = ['data/mimic_test.tfrecords']


    raw_dataset = tf.data.TFRecordDataset(filename)
    for raw_record in raw_dataset:

        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

        race = example.features.feature['race'].int64_list.value[0]
        age = example.features.feature['age'].int64_list.value[0]
        gender = example.features.feature['gender'].int64_list.value[0]

        label = []
        label.append(1 if example.features.feature['Atelectasis'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Cardiomegaly'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Consolidation'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Edema'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Enlarged Cardiomediastinum'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Fracture'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Lung Lesion'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Lung Opacity'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['No Finding'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Pleural Effusion'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Pleural Other'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Pneumonia'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Pneumothorax'].float_list.value[0] == 1 else 0)
        label.append(1 if example.features.feature['Support Devices'].float_list.value[0] == 1 else 0)

        if (types == 'race'):
            if (race == 0):
                x.append([1, 0, 0])
            elif (race == 1):
                x.append([0, 1, 0])
            elif (race == 4):
                x.append([0, 0, 1])
            else:
                continue
        elif (types == 'age'):
            if (age == 0 or age == 1):
                x.append([1, 0, 0, 0])
            elif (age == 2):
                x.append([0, 1, 0, 0])
            elif (age == 3):
                x.append([0, 0, 1, 0])
            elif (age == 4):
                x.append([0, 0, 0, 1])
            else:
                continue
        elif (types == 'gender'):
            if (gender == 0):
                x.append([1, 0])
            elif (gender == 1):
                x.append([0, 1])
            else:
                continue
        else:
            continue

        y.append(label)

    return np.array(x), np.array(y)

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

Labels_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

x_train, y_train = get_data_permutaitontest('train', 'race')

clf = OneVsRestClassifier(LogisticRegression(random_state=2021)).fit(x_train, y_train)

x_test, y_test = get_data_permutaitontest('test', 'race')

preds = clf.predict_proba(x_test)

num = []
for i in range(14):
    num.append(np.unique(preds[:, i]))

for j in range(len(preds)):
    for i in range(14):
        if (preds[j][i] >= np.median(num[i])):
            preds[j][i] = 1
        else:
            preds[j][i] = 0

result_race = classification_report(y_test, preds, output_dict=True)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.22      0.86      0.35     12314
           1       0.21      0.23      0.22     12152
           2       0.05      0.86      0.09      2770
           3       0.13      0.85      0.23      7427
           4       0.03      0.86      0.07      1951
           5       0.02      0.93      0.04      1131
           6       0.03      0.86      0.06      1653
           7       0.24      0.84      0.37     13457
           8       0.38      0.27      0.31     18362
           9       0.27      0.89      0.41     14679
          10       0.01      0.89      0.02       504
          11       0.07      0.83      0.13      4107
          12       0.05      0.88      0.10      2818
          13       0.32      0.86      0.47     17870

   micro avg       0.13      0.69      0.22    111195
   macro avg       0.15      0.78      0.21    111195
weighted avg       0.24      0.69      0.32    111195
 samples avg       0.15   

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

Labels_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

x_train, y_train = get_data_permutaitontest('train', 'age')

clf = OneVsRestClassifier(LogisticRegression(random_state=2021)).fit(x_train, y_train)

x_test, y_test = get_data_permutaitontest('test', 'age')

preds = clf.predict_proba(x_test)

num = []
for i in range(14):
    num.append(np.unique(preds[:, i]))

for j in range(len(preds)):
    for i in range(14):
        if (preds[j][i] >= np.median(num[i])):
            preds[j][i] = 1
        else:
            preds[j][i] = 0

result_age = classification_report(y_test, preds, output_dict=True)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.24      0.67      0.35     12314
           1       0.24      0.69      0.36     12152
           2       0.05      0.64      0.09      2770
           3       0.15      0.72      0.25      7427
           4       0.03      0.75      0.07      1951
           5       0.02      0.65      0.04      1131
           6       0.03      0.69      0.06      1653
           7       0.25      0.64      0.36     13457
           8       0.41      0.54      0.47     18362
           9       0.31      0.72      0.43     14679
          10       0.01      0.72      0.02       504
          11       0.07      0.61      0.13      4107
          12       0.05      0.59      0.10      2818
          13       0.31      0.61      0.41     17870

   micro avg       0.15      0.64      0.24    111195
   macro avg       0.16      0.66      0.22    111195
weighted avg       0.26      0.64      0.35    111195
 samples avg       0.18   

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

Labels_diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

x_train, y_train = get_data_permutaitontest('train', 'gender')

clf = OneVsRestClassifier(LogisticRegression(random_state=2021)).fit(x_train, y_train)

x_test, y_test = get_data_permutaitontest('test', 'gender')

preds = clf.predict_proba(x_test)

num = []
for i in range(14):
    num.append(np.unique(preds[:, i]))

for j in range(len(preds)):
    for i in range(14):
        if (preds[j][i] >= np.median(num[i])):
            preds[j][i] = 1
        else:
            preds[j][i] = 0

result_gender = classification_report(y_test, preds, output_dict=True)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.22      0.55      0.32     12314
           1       0.20      0.51      0.29     12152
           2       0.05      0.54      0.09      2770
           3       0.12      0.50      0.20      7427
           4       0.04      0.56      0.07      1951
           5       0.02      0.57      0.04      1131
           6       0.03      0.49      0.06      1653
           7       0.23      0.53      0.33     13457
           8       0.33      0.50      0.40     18362
           9       0.25      0.53      0.34     14679
          10       0.01      0.58      0.02       504
          11       0.07      0.55      0.13      4107
          12       0.06      0.62      0.10      2818
          13       0.33      0.56      0.41     17870

   micro avg       0.14      0.53      0.22    111195
   macro avg       0.14      0.54      0.20    111195
weighted avg       0.23      0.53      0.31    111195
 samples avg       0.15   

In [21]:
from sklearn.metrics import f1_score
np.random.seed(2021)
x_test, y_test = get_data_permutaitontest('test')
length = len(y_test)
a = []
for i in range(100000):
    guess = np.copy(y_test)
    np.random.shuffle(guess)

    a.append(classification_report(y_test, guess, output_dict=True))

In [ ]:
for i in list(a[0].keys()):
    print(i)
    b = list(map(lambda x: x[i]['f1-score'], a))
    c = plt.hist(b, bins=100, color='limegreen')
    plt.axvline(np.percentile(b, 95), color='r', linestyle='dashed', linewidth=1)
    plt.axvline(result_race['weighted avg']['f1-score'], color='g', linestyle='dashed', linewidth=1)
    plt.axvline(result_age['weighted avg']['f1-score'], color='b', linestyle='dashed', linewidth=1)
    plt.axvline(result_gender['weighted avg']['f1-score'], color='y', linestyle='dashed', linewidth=1)
    plt.show()